In [1]:
import numpy as np
import pandas as pd

In [2]:
df_bak = pd.read_excel('../01-Data/ordens_pre_process_revisado.xlsx')

In [3]:
df_caixas_tipo = pd.read_excel('../01-Data/Dados - desafio disciplina RPVMM.xlsx', sheet_name='Tipos de caixa')

In [4]:
df_routes = df_bak[['Rota', 'Capacidade']].drop_duplicates()
# df_routes = df_routes.set_index('Rota')
df_routes

,Rota,Capacidade
0,1,inf
3,3,inf
9,10,2.720000e+10
14,8,4.000000e+10
26,6,2.720000e+10
45,13,2.720000e+10
71,11,2.720000e+10
76,12,1.600000e+10
116,5,2.720000e+10
147,15,4.000000e+10


In [5]:
df_caixas_tipo.columns = df_caixas_tipo.columns.str.replace(' ', '_')
df_caixas_tipo['volume_util_caixa'] = df_caixas_tipo['Comprimento'] * df_caixas_tipo['Largura'] * df_caixas_tipo['Altura'] * df_caixas_tipo['Fator_de_ocupação']
df_caixas_tipo

,Tipo,Tipo_de_buffer,Comprimento,Largura,Altura,Fator_de_ocupação,Peças_max,Peso_max_(kg),volume_util_caixa
0,Caixa 1,CX,380,285,285,0.9,170,23,27778950.0
1,Caixa 2,CX,586,436,272,1.2,170,23,83393894.4
2,Caixa 3,CX,728,427,270,0.9,170,23,75538008.0
3,Caixa 1,EN,380,285,285,0.9,512,23,27778950.0
4,Caixa 2,EN,586,436,272,1.2,512,23,83393894.4
5,Caixa 3,EN,728,427,270,0.9,512,23,75538008.0


In [6]:
df_caixas_padrao = df_caixas_tipo.loc[[1, 5]]
df_caixas_padrao

,Tipo,Tipo_de_buffer,Comprimento,Largura,Altura,Fator_de_ocupação,Peças_max,Peso_max_(kg),volume_util_caixa
1,Caixa 2,CX,586,436,272,1.2,170,23,83393894.4
5,Caixa 3,EN,728,427,270,0.9,512,23,75538008.0


In [7]:
df = df_bak.drop(['Unnamed: 0'], axis=1)
df.columns = df.columns.str.replace(' ', '_')
df["Volume_total"] = df["Volume_unit"] * df["Peças"]
df["Peso_total"] = df["Peso_unit"] * df["Peças"]

In [8]:
# df = df.query("chave_loja_buffer_onda in ('10_EN_3', '28_EN_4', '16_EN_3', '2_EN_3', '31_EN_3', '33_EN_3', '8_EN_4', '10_EN_4', '9_EN_4', '3_EN_3', '25_EN_4', '18_EN_4', '48_EN_4', '8_EN_3', '10_EN_1', '23_EN_3', '11_EN_4', '3_EN_4', '2_EN_4', '9_EN_3')")
df = df.query('chave_loja_buffer_onda in ("16_EN_3","9_EN_4","33_EN_3")')

In [9]:
print(df['Inner'].unique())

[1]


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1041 entries, 33868 to 59790
Data columns (total 23 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   Ordem_de_produção          1041 non-null   int64         
 1   Data_da_ordem_de_produção  1041 non-null   datetime64[ns]
 2   Produto                    1041 non-null   int64         
 3   Comprimento_unit           1041 non-null   int64         
 4   Largura_unit               1041 non-null   int64         
 5   Altura_unit                1041 non-null   int64         
 6   Peso_unit                  1041 non-null   float64       
 7   Peças                      1041 non-null   int64         
 8   Caixa_padrão               1041 non-null   object        
 9   Loja                       1041 non-null   int64         
 10  Classe_de_onda             1041 non-null   int64         
 11  Tipo_de_buffer             1041 non-null   object        
 12  Item_p

In [11]:
df["Peças"].sum()   

5343

In [12]:
df.columns

Index(['Ordem_de_produção', 'Data_da_ordem_de_produção', 'Produto',
       'Comprimento_unit', 'Largura_unit', 'Altura_unit', 'Peso_unit', 'Peças',
       'Caixa_padrão', 'Loja', 'Classe_de_onda', 'Tipo_de_buffer', 'Item_pai',
       'Cor', 'Tamanho', 'Inner', 'Rota', 'Capacidade', 'pai_cor',
       'chave_loja_buffer_onda', 'Volume_unit', 'Volume_total', 'Peso_total'],
      dtype='object')

In [22]:
def first_fit_decreasing(df, bin_restrictions, route_restrictions):
    # Sort items in decreasing order of volume and increasing order of date (FIFO)
    df_sorted = df.copy()
    df_sorted['Inv_Volume_total'] = (1 / df_sorted['Volume_total']).replace(np.inf, 0)
    df_sorted = df_sorted.sort_values(by=['Data_da_ordem_de_produção', 'Inv_Volume_total', 'pai_cor'], ascending=True)
    
    # Initialize bins list
    bins = []
    leftover_items = pd.DataFrame(columns=df_sorted.columns)  # Initialize leftover items DataFrame
    
    # Flag to track if a split was made
    split_made = True
    items_to_drop = []
    df_sorted = df_sorted.reset_index(drop=True)
    items_to_add = []

    while len(df_sorted) != 0:
        split_made = False
        df_sorted = df_sorted[~df_sorted.index.isin(items_to_drop)]

        # Add needed items to df_sorted
        if len(items_to_add) != 0:
            df_sorted = pd.concat([df_sorted, pd.DataFrame(items_to_add)])
        items_to_add = []
        df_sorted = df_sorted.reset_index(drop=True)
        items_to_drop = []
        
        # Make a copy of the sorted DataFrame
        df_sorted_copy = df_sorted.copy()
        
        # Iterate over items in the copy of the sorted DataFrame
        for index, item in df_sorted_copy.iterrows():
            # Flag to check if the item is placed
            placed = False
            
            # Try to place the item in existing bins
            for bin in bins:
                if (bin['Volume'] + item['Volume_total']) <= bin["vol_limit"] \
                    and (bin['Peso_total'] + item['Peso_total']) <= 23 \
                    and bin['Amt.Items'] + item['Peças'] <= bin["item_limit"] \
                    and item["Loja"] == bin["Loja"] \
                    and item["Rota"] == bin["Rota"] \
                    and item["Tipo_de_buffer"] == bin["Buffer"] \
                    and item["Classe_de_onda"] == bin["Wave"]:
                    
                    bin['Items'].append(index)
                    bin['Volume'] += item['Volume_total']
                    bin['Peso_total'] += item['Peso_total']
                    bin['Amt.Items'] += item['Peças']

                    # Remove the item from the original DataFrame
                    items_to_drop.append(index)
                    placed = True
                    break
            
            # If the item couldn't be placed entirely, reduce it by "inner" means until it fits into a bin
            while not placed and item['Peças'] > item['Inner']:
                # Reduce the item to its inner size
                detached_portion = item.copy()
                detached_portion['Peças'] = item['Inner']
                detached_portion['Volume_total'] = item['Volume_unit'] * item['Inner']
                detached_portion['Peso_total'] = item['Peso_unit'] * item['Inner']
                
                # Update the remaining portion of the item
                item['Peças'] -= item['Inner']
                item['Volume_total'] -= item['Volume_unit'] * item['Inner']
                item['Peso_total'] -= item['Peso_unit'] * item['Inner']
                
                # Concatenate the reduced portion back to the original DataFrame
                items_to_add.append(detached_portion)
                split_made = True
                
                # Try to place the reduced item in existing bins
                for bin in bins:
                    if (bin['Volume'] + item['Volume_total']) <= bin["vol_limit"] \
                        and (bin['Peso_total'] + item['Peso_total']) <= 23 \
                        and bin['Amt.Items'] + item['Peças'] <= bin["item_limit"] \
                        and item["Loja"] == bin["Loja"] \
                        and item["Rota"] == bin["Rota"] \
                        and item["Tipo_de_buffer"] == bin["Buffer"] \
                        and item["Classe_de_onda"] == bin["Wave"]:
                        
                        bin['Items'].append(index)  # Append index of the appended reduced item
                        bin['Volume'] += item['Volume_total']
                        bin['Peso_total'] += item['Peso_total']
                        bin['Amt.Items'] += item['Peças']

                        items_to_drop.append(index) # Remove the reduced item from the original DataFrame
                        placed = True
                        break
            
            # If the item still couldn't be placed, create a new bin
            if not placed:
                # Check if creating a new bin exceeds route capacity
                if len([bin for bin in bins if bin['Rota'] == item["Rota"]]) >= route_restrictions[route_restrictions["Rota"] == item["Rota"]]["Capacidade"].values[0]:
                    leftover_items = leftover_items.append(item)  # Add item to leftover items DataFrame
                else:
                    bins.append({
                        'Items': [index],
                        'Volume': item['Volume_total'],
                        'Peso_total': item['Peso_total'],
                        'Amt.Items': item['Peças'],
                        "Loja": item["Loja"],
                        "Rota": item["Rota"],
                        "Buffer": item["Tipo_de_buffer"],
                        "Wave": item["Classe_de_onda"],
                        "vol_limit": bin_restrictions[bin_restrictions["Tipo_de_buffer"] == item["Tipo_de_buffer"]]["volume_util_caixa"].values[0],
                        "item_limit": bin_restrictions[bin_restrictions["Tipo_de_buffer"] == item["Tipo_de_buffer"]]["Peças_max"].values[0]
                    })
                    items_to_drop.append(index)
                    placed = True
                    break
    
    return bins, leftover_items


In [23]:
def add_item_loja_column(df):
    # Group by 'Loja' and 'Produto' and use cumcount to assign a sequential number starting from 1
    df['item_loja'] = df.groupby(['Loja']).cumcount() + 1
    return df


In [24]:
df_caixas_padrao

,Tipo,Tipo_de_buffer,Comprimento,Largura,Altura,Fator_de_ocupação,Peças_max,Peso_max_(kg),volume_util_caixa
1,Caixa 2,CX,586,436,272,1.2,170,23,83393894.4
5,Caixa 3,EN,728,427,270,0.9,512,23,75538008.0


In [25]:
backlog = df.copy()
backlog = backlog.reset_index(drop=True)
backlog = add_item_loja_column(backlog)
packed_bins, leftovers = first_fit_decreasing(df, bin_restrictions=df_caixas_padrao, route_restrictions=df_routes)
print("Packed bins (new method):")
for i, bin in enumerate(packed_bins):
    print(f"Bin {i+1}: Amt. Items {bin['Amt.Items']} (Total Volume: {bin['Volume']} cubic mm | Total Weight: {bin['Peso_total'] :.4f}kg)")

print(f"\nLeftover items: {leftovers['Peças'].sum()} items")


Packed bins (new method):
Bin 1: Amt. Items 25 (Total Volume: 75520446 cubic mm | Total Weight: 7.7330kg)
Bin 2: Amt. Items 22 (Total Volume: 75051750 cubic mm | Total Weight: 7.0930kg)
Bin 3: Amt. Items 20 (Total Volume: 74832759 cubic mm | Total Weight: 4.3340kg)
Bin 4: Amt. Items 22 (Total Volume: 75289616 cubic mm | Total Weight: 4.7270kg)
Bin 5: Amt. Items 16 (Total Volume: 75334857 cubic mm | Total Weight: 3.6620kg)
Bin 6: Amt. Items 13 (Total Volume: 74927770 cubic mm | Total Weight: 3.8650kg)
Bin 7: Amt. Items 11 (Total Volume: 74927988 cubic mm | Total Weight: 3.7960kg)
Bin 8: Amt. Items 12 (Total Volume: 75484293 cubic mm | Total Weight: 4.7450kg)
Bin 9: Amt. Items 15 (Total Volume: 75473590 cubic mm | Total Weight: 3.5990kg)
Bin 10: Amt. Items 27 (Total Volume: 75426883 cubic mm | Total Weight: 5.4170kg)
Bin 11: Amt. Items 26 (Total Volume: 75298308 cubic mm | Total Weight: 5.6100kg)
Bin 12: Amt. Items 12 (Total Volume: 74798400 cubic mm | Total Weight: 4.7450kg)
Bin 13: Amt

In [26]:
import pandas as pd

def postprocess_bins(bins, backlog_df):
    # Initialize a list to store the rows for the final DataFrame
    rows = []

    # Iterate over the bins to extract item information
    for bin_index, bin in enumerate(bins):
        for item_index in bin['Items']:
            item = backlog_df.loc[item_index].copy()
            item['Bin'] = bin_index  # Add the bin index
            item['Amt_in_bin'] = item['Peças']  # Add the amount in the bin
            rows.append(item)
    
    # Create a DataFrame from the rows
    result_df = pd.DataFrame(rows)
    
    return result_df

In [27]:
print(backlog['item_loja'].unique())

[  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54
  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72
  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90
  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107 108
 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126
 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144
 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162
 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180
 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198
 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216
 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234
 235 236 237 238 239 240 241 242 243 244 245 246 24

In [28]:
result_df = postprocess_bins(packed_bins, backlog).reset_index(drop=True)

# Print the resulting DataFrame
result_df[['Ordem_de_produção', 'Data_da_ordem_de_produção', 'Produto', 'item_loja', 'Loja', 'Bin', 'Item_pai', 'Amt_in_bin']]

,Ordem_de_produção,Data_da_ordem_de_produção,Produto,item_loja,Loja,Bin,Item_pai,Amt_in_bin
0,40,2023-10-30,847254655,1,9,0,40,3
1,40,2023-10-30,847254655,1,9,0,40,3
2,915,2023-10-30,847254621,2,9,0,40,3
3,4051,2023-10-30,847254663,3,9,0,40,2
4,8788,2023-10-30,847254680,4,9,0,40,2
...,...,...,...,...,...,...,...,...
3317,761,2023-10-31,547520173,1,33,325,1252,1
3318,761,2023-10-31,547520181,2,33,325,1252,1
3319,761,2023-10-31,557880308,3,33,325,1252,1
3320,761,2023-10-31,864661421,4,33,325,1252,2


In [30]:
df_sorted = df.copy()
df_sorted['Inv_Volume_total'] = (1 / df_sorted['Volume_total']).replace(np.inf, 0)
df_sorted = df_sorted.sort_values(by=['Data_da_ordem_de_produção', 'Inv_Volume_total', 'pai_cor'], ascending=True)
df_sorted

,Ordem_de_produção,Data_da_ordem_de_produção,Produto,Comprimento_unit,Largura_unit,Altura_unit,Peso_unit,Peças,Caixa_padrão,Loja,...,Tamanho,Inner,Rota,Capacidade,pai_cor,chave_loja_buffer_onda,Volume_unit,Volume_total,Peso_total,Inv_Volume_total
33877,6473,2023-10-30,616819153,433,277,26,0.420,9,CARTON_M,9,...,21,1,1,inf,40_99.0,9_EN_4,3118466,28066194,3.780,3.563005e-08
33880,7717,2023-10-30,616819188,400,280,30,0.320,7,CARTON_M,9,...,24,1,1,inf,40_99.0,9_EN_4,3360000,23520000,2.240,4.251701e-08
33873,11115,2023-10-30,847254647,389,217,32,0.303,6,CARTON_M,9,...,24,1,1,inf,40_28.0,9_EN_4,2701216,16207296,1.818,6.170061e-08
33872,8438,2023-10-30,847254639,432,248,32,0.310,4,CARTON_M,9,...,39,1,1,inf,40_28.0,9_EN_4,3428352,13713408,1.240,7.292133e-08
33869,915,2023-10-30,847254621,445,242,33,0.323,3,CARTON_M,9,...,31,1,1,inf,40_28.0,9_EN_4,3553770,10661310,0.969,9.379710e-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57772,10365,2023-11-21,549983673,363,238,16,0.232,3,CARTON_M,16,...,3,1,1,inf,233_3.0,16_EN_3,1382304,4146912,0.696,2.411433e-07
59785,9664,2023-11-21,861665881,480,354,23,0.320,1,CARTON_M,9,...,6,1,1,inf,718_281.0,9_EN_4,3908160,3908160,0.320,2.558749e-07
59782,4116,2023-11-21,861665944,465,355,21,0.267,1,CARTON_M,9,...,13,1,1,inf,718_281.0,9_EN_4,3466575,3466575,0.267,2.884692e-07
58413,761,2023-11-21,869448821,295,247,19,0.167,2,CARTON_M,33,...,6,1,1,inf,460_137.0,33_EN_3,1384435,2768870,0.334,3.611582e-07


In [21]:
df_sorted['Rota'].value_counts()

Rota
1    1041
Name: count, dtype: int64